<a href="https://colab.research.google.com/github/lefkothea1/NLP_2/blob/main/Copy_of_NLP_ass2_ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#to select GPU: Edit → Notebook Settings

#loading data from github repo

import pandas as pd
url_diagn_test = 'https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-subset-diagnostic-tests.csv'
url_test = 'https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-test.csv'
url_train ='https://raw.githubusercontent.com/lefkothea1/NLP_2/main/data/Assignment%202_IntrotoNLP2022_data/olid-train.csv'

diagnostic_test = pd.read_csv(url_test)
train = pd.read_csv(url_train)
test = pd.read_csv(url_test)

print(train.shape)
print(train.head())


In [3]:
#should i install it every time?-yes
!pip install simpletransformers 

     |████████████████████████████████| 249 kB 5.3 MB/s 
     |████████████████████████████████| 10.1 MB 42.7 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 35.9 MB/s 
     |████████████████████████████████| 6.6 MB 36.4 MB/s 
     |████████████████████████████████| 325 kB 48.4 MB/s 
     |████████████████████████████████| 4.0 MB 19.3 MB/s 
     |████████████████████████████████| 1.8 MB 35.5 MB/s 
     |████████████████████████████████| 596 kB 34.6 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 41.6 MB/s 
     |████████████████████████████████| 181 kB 50.6 MB/s 
     |████████████████████████████████| 144 kB 20.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 136 kB 32.0 MB/s 
     |████████████████████████████████| 1.1 MB 40.7 MB/s 
     |████████████████████████████████| 212 kB 45.9 MB/s 
     |██████████████

In [4]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [8]:
# Optional model configuration-using best teams args -from paper
model_args = ClassificationArgs()
model_args.num_train_epochs=2
model_args.max_sentence_length=64

NameError: ignored

In [6]:
#loading neccessary pretrained(?) model
model = ClassificationModel('bert', 'bert-base-cased')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [7]:
#training/ finetuning on OUR training data-takes time! DONE WITHOUT MODEL ARGS CURRENTLY
model.train_model(train)

  0%|          | 0/13240 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1655 [00:00<?, ?it/s]

(1655, 0.494665979114544)